In [1]:
import pandas as pd
import os
!pip install pykrige

In [ ]:
delhi

In [2]:
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging


In [3]:
nlags=20

#Get the list of columns that have NaN values
columns_with_nan = delhi.columns[delhi.isna().any()].tolist()
columns_with_nan

In [4]:
import pandas as pd
from pykrige.ok import OrdinaryKriging
import numpy as np

def estimate_pollutant(df, date_column, nlags=20):
    # Create a DataFrame with only latitude, longitude, and the specified date column
    df1 = df[['longitude', 'latitude', date_column]]

    # Create a DataFrame with latitude and longitude where the specified date column's values are NaN
    nan_locations_df = df1[df1[date_column].isna()][['longitude', 'latitude']]

    # Create a DataFrame with latitude, longitude, and the specified date column excluding NaN values
    pollutant_df = df1.dropna(subset=[date_column])[['longitude', 'latitude', date_column]]

    # Prepare the data for Ordinary Kriging
    lons = np.array(pollutant_df['longitude'])
    lats = np.array(pollutant_df['latitude'])
    zdata = np.array(pollutant_df[date_column])

    # Perform Ordinary Kriging
    OK = OrdinaryKriging(lons, lats, zdata, variogram_model='linear', verbose=True, enable_plotting=False, nlags=nlags)
    grid_lon = np.array(nan_locations_df["longitude"])
    grid_lat = np.array(nan_locations_df["latitude"])
    z1, ss1 = OK.execute('grid', grid_lon, grid_lat)

    # Fill in NaN values in the specified date column with the interpolated values
    nan_locations_df[date_column] = z1[0]

    # Merge the two DataFrames based on their indices
    merged_df = pd.concat([pollutant_df, nan_locations_df], ignore_index=False)

    # Sort the DataFrame by ascending index
    merged_df = merged_df.sort_index(ascending=True)

    return merged_df

In [ ]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_Co/Chosen_state_2023_sep_merge_sentinel5p_Co/Delhi_2023_sep_Co.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_Co/After_krigging_2023_sep_sentinal5p_Co/Delhi_2023_sep_Co_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 4.3374039820812165e-06
Nugget: 7.171990892976116e-06 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 7.033567795649457e-06
Nugget: 1.8700388140737386e-05 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 6.586362838123647e-05
Nugget: 3.2722573031145425e-05 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...



In [47]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-26,2023-09-27,2023-09-28,2023-09-29
0,28.551,76.849,0.120806,0.138337,0.121830,0.125714,0.124514,0.131611,0.130002,0.133719,...,0.116448,0.124113,0.128183,0.117908,0.126800,0.129408,0.124271,0.125626,0.122071,0.133300
1,28.561,76.849,0.123113,0.137685,0.122348,0.126087,0.132372,0.128111,0.129450,0.134803,...,0.118284,0.120593,0.115197,0.125402,0.129639,0.130577,0.124705,0.125119,0.123917,0.127497
2,28.571,76.849,0.119602,0.138337,0.124811,0.122651,0.126359,0.138333,0.138329,0.135462,...,0.119477,0.124305,0.127623,0.114613,0.126278,0.127863,0.121576,0.137131,0.122933,0.126219
3,28.580,76.849,0.143472,0.138337,0.126294,0.123821,0.127240,0.126820,0.137157,0.135462,...,0.127097,0.120813,0.137923,0.113922,0.127896,0.126723,0.128667,0.135450,0.124115,0.135560
4,28.551,76.860,0.120626,0.135761,0.124279,0.122046,0.129242,0.131650,0.122101,0.122350,...,0.113653,0.122268,0.127811,0.114778,0.119219,0.126112,0.125284,0.126075,0.123255,0.122441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,28.688,77.327,0.126005,0.135732,0.131573,0.128185,0.129889,0.129930,0.134419,0.115280,...,0.116320,0.126775,0.112370,0.121682,0.134373,0.129551,0.126508,0.135783,0.118817,0.131802
1346,28.512,77.337,0.122849,0.133906,0.124334,0.125453,0.128707,0.127801,0.132237,0.139142,...,0.116132,0.123082,0.126620,0.119282,0.129121,0.119854,0.123538,0.137503,0.124085,0.128500
1347,28.522,77.337,0.120734,0.159084,0.124543,0.128229,0.128339,0.128059,0.130902,0.135073,...,0.117879,0.124561,0.132376,0.117276,0.123837,0.120341,0.125068,0.136494,0.122596,0.137517
1348,28.610,77.337,0.119194,0.137661,0.126031,0.121886,0.129947,0.131101,0.130902,0.118528,...,0.118249,0.123457,0.129906,0.114437,0.124167,0.120333,0.121093,0.126896,0.119142,0.123108


In [ ]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_Co/Chosen_state_2023_sep_merge_sentinel5p_Co/WB_2023_sep_Co.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_Co/After_krigging_2023_sep_sentinal5p_Co/Wb_2023_sep_Co_final.csv",index = False)